In [1]:
%matplotlib inline

In [26]:
import cv2
import os
import math
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from sklearn import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten,MaxPooling2D,Dropout
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (
    EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
)
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from tensorflow.keras.optimizers import Adam,SGD
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from mostCommon import most_common,patient_score

In [27]:
IMAGE_WIDTH=224
IMAGE_HEIGHT=224

In [28]:
#Load all patients 
my_data = pd.read_csv('../data/Fifteen_Frames_Per_patient.csv')
patients_data=pd.read_csv('../data/videos_data.csv')
data_array = my_data.to_numpy()
patients_array=patients_data["FileName"].to_numpy()
images_path = '../data/Fifteen_Frames_Per_patient/'
no_of_test_patients=math.floor(patients_array.size*0.2) 
images_data = []
models = []

df=my_data
data_array=df.to_numpy()
for i in data_array:
    img = cv2.imread(images_path + i[0] + '.' + i[2], cv2.IMREAD_GRAYSCALE)
    img=  cv2.equalizeHist(img)
    img = cv2.resize(img, (IMAGE_WIDTH, IMAGE_HEIGHT))
    img_2d = img.reshape(IMAGE_WIDTH ,IMAGE_HEIGHT,1)
    images_data.append(img_2d)

X=np.asarray(images_data)
y = np.asarray(my_data['Label'])
print(Counter(y))

Counter({'regular': 840, 'pneumonia': 540, 'covid': 450})


In [29]:
testPatient_indexes= [1,2,12,14,15,22,
                      33,34,37,42,43,52,53,
                      92,95,96,99,101,102,104,107,110,115,118]


In [30]:
#getting frame indexes of each patient
frames_index=[]
for i in testPatient_indexes:
  j=i*15
  z=j+15
  while j<z:
    frames_index.append(j)
    j+=1

# Adding the frames to X_test
X_test=np.take(X,frames_index,axis=0)
y_test=np.take(y,frames_index,axis=0)
# Removing the frames from X_train
X_train=np.delete(X,frames_index,axis=0)
y_train=np.delete(y,frames_index,axis=0)

# Scale the pixels
X_train=X_train/255.0
X_test=X_test/255.0

In [31]:
#Applying SMOTE
oversample = SMOTE()
X_train=np.resize(X_train,(X_train.shape[0],50176))
X_train,y_train=oversample.fit_resample(X_train, y_train)
counter=Counter(y_train)
print(counter)

Counter({'covid': 675, 'pneumonia': 675, 'regular': 675})


In [32]:
print('X_train: '+str(X_train.shape))
print('y_train: '+str(y_train.shape))
print('X_test: '+ str(X_test.shape))
print('y_test: '+ str(y_test.shape))

X_train: (2025, 50176)
y_train: (2025,)
X_test: (360, 224, 224, 1)
y_test: (360,)


In [33]:
model=RandomForestClassifier(max_depth=10000, random_state=42)
model.fit(X_train,y_train)

RandomForestClassifier(max_depth=10000, random_state=42)

In [34]:
X_test=np.resize(X_test,(X_test.shape[0],50176))
y_pred=model.predict(X_test)

In [35]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred, digits=3))
patient_score(y_test,y_pred)

[[ 85   0   5]
 [  0  95  10]
 [  0  23 142]]
              precision    recall  f1-score   support

       covid      1.000     0.944     0.971        90
   pneumonia      0.805     0.905     0.852       105
     regular      0.904     0.861     0.882       165

    accuracy                          0.894       360
   macro avg      0.903     0.903     0.902       360
weighted avg      0.899     0.894     0.896       360

[[6 0 0]
 [0 7 0]
 [0 2 9]]
              precision    recall  f1-score   support

       covid      1.000     1.000     1.000         6
   pneumonia      0.778     1.000     0.875         7
     regular      1.000     0.818     0.900        11

    accuracy                          0.917        24
   macro avg      0.926     0.939     0.925        24
weighted avg      0.935     0.917     0.918        24



In [36]:
>>> from sklearn.model_selection import cross_val_score

scores=cross_val_score(model, X_test, y_test, cv=5)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.88 accuracy with a standard deviation of 0.13


In [37]:
IMAGE_WIDTH=224
IMAGE_HEIGHT=224
#Load all patients 
my_data = pd.read_csv('../data/Fifteen_Frames_Per_patient.csv')
patients_data=pd.read_csv('../data/videos_data.csv')
data_array = my_data.to_numpy()
patients_array=patients_data["FileName"].to_numpy()
images_path = '../data/Fifteen_Frames_Per_patient/'
no_of_test_patients=math.floor(patients_array.size*0.2) 
images_data = []
models = []

df=my_data
data_array=df.to_numpy()
for i in data_array:
    img = cv2.imread(images_path + i[0] + '.' + i[2], cv2.IMREAD_GRAYSCALE)
    img=  cv2.equalizeHist(img)
    img = cv2.resize(img, (IMAGE_WIDTH, IMAGE_HEIGHT))
    img_2d = img.reshape(IMAGE_WIDTH ,IMAGE_HEIGHT,1)
    images_data.append(img_2d)

X=np.asarray(images_data)
y = np.asarray(my_data['Label'])
print(Counter(y))

testPatient_indexes= [1,2,12,14,15,22,
                      33,34,37,42,43,52,53,
                      92,95,96,99,101,102,104,107,110,115,118]
#getting frame indexes of each patient
frames_index=[]
for i in testPatient_indexes:
  j=i*15
  z=j+15
  while j<z:
    frames_index.append(j)
    j+=1

# Adding the frames to X_test
X_test=np.take(X,frames_index,axis=0)
y_test=np.take(y,frames_index,axis=0)
# Removing the frames from X_train
X_train=np.delete(X,frames_index,axis=0)
y_train=np.delete(y,frames_index,axis=0)

# Scale the pixels
X_train=X_train/255.0
X_test=X_test/255.0
X_train=np.resize(X_train,(X_train.shape[0],50176))
print('X_train: '+str(X_train.shape))
print('y_train: '+str(y_train.shape))
print('X_test: '+ str(X_test.shape))
print('y_test: '+ str(y_test.shape))
model=RandomForestClassifier(max_depth=10000, random_state=42)
model.fit(X_train,y_train)
X_test=np.resize(X_test,(X_test.shape[0],50176))
y_pred=model.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred, digits=3))
patient_score(y_test,y_pred)
from sklearn.model_selection import cross_val_score

scores=cross_val_score(model, X_train, y_train, cv=5)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Counter({'regular': 840, 'pneumonia': 540, 'covid': 450})
X_train: (1470, 50176)
y_train: (1470,)
X_test: (360, 224, 224, 1)
y_test: (360,)
[[ 87   0   3]
 [  0 104   1]
 [  1  22 142]]
              precision    recall  f1-score   support

       covid      0.989     0.967     0.978        90
   pneumonia      0.825     0.990     0.900       105
     regular      0.973     0.861     0.913       165

    accuracy                          0.925       360
   macro avg      0.929     0.939     0.930       360
weighted avg      0.934     0.925     0.926       360

[[ 6  0  0]
 [ 0  7  0]
 [ 0  1 10]]
              precision    recall  f1-score   support

       covid      1.000     1.000     1.000         6
   pneumonia      0.875     1.000     0.933         7
     regular      1.000     0.909     0.952        11

    accuracy                          0.958        24
   macro avg      0.958     0.970     0.962        24
weighted avg      0.964     0.958     0.959        24

0.71 accuracy w

In [38]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
IMAGE_WIDTH=224
IMAGE_HEIGHT=224
#Load all patients 
my_data = pd.read_csv('../data/Fifteen_Frames_Per_patient.csv')
patients_data=pd.read_csv('../data/videos_data.csv')
data_array = my_data.to_numpy()
patients_array=patients_data["FileName"].to_numpy()
images_path = '../data/Fifteen_Frames_Per_patient/'
no_of_test_patients=math.floor(patients_array.size*0.2) 
images_data = []
models = []

df=my_data
data_array=df.to_numpy()
for i in data_array:
    img = cv2.imread(images_path + i[0] + '.' + i[2], cv2.IMREAD_GRAYSCALE)
    img=  cv2.equalizeHist(img)
    img = cv2.resize(img, (IMAGE_WIDTH, IMAGE_HEIGHT))
    img_1d = img.reshape(224 * 224)
    images_data.append(img_1d)

X=np.asarray(images_data)
y = np.asarray(my_data['Label'])
print(Counter(y))
scaler = StandardScaler()
# Fit on training set only
scaler.fit(X)
# Apply transform on both training and test set

X = scaler.transform(X)
pca = PCA(.95)
pca.fit(X)
X = pca.transform(X)

testPatient_indexes= [1,2,12,14,15,22,
                      33,34,37,42,43,52,53,
                      92,95,96,99,101,102,104,107,110,115,118]
#getting frame indexes of each patient
frames_index=[]
for i in testPatient_indexes:
  j=i*15
  z=j+15
  while j<z:
    frames_index.append(j)
    j+=1

# Adding the frames to X_test
X_test=np.take(X,frames_index,axis=0)
y_test=np.take(y,frames_index,axis=0)
# Removing the frames from X_train
X_train=np.delete(X,frames_index,axis=0)
y_train=np.delete(y,frames_index,axis=0)


print('X_train: '+str(X_train.shape))
print('y_train: '+str(y_train.shape))
print('X_test: '+ str(X_test.shape))
print('y_test: '+ str(y_test.shape))
model=RandomForestClassifier(max_depth=10000, random_state=42)
model.fit(X_train,y_train)

y_pred=model.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred, digits=3))
patient_score(y_test,y_pred)
from sklearn.model_selection import cross_val_score

scores=cross_val_score(model, X_train, y_train, cv=5)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Counter({'regular': 840, 'pneumonia': 540, 'covid': 450})
X_train: (1470, 116)
y_train: (1470,)
X_test: (360, 116)
y_test: (360,)
[[ 66  12  12]
 [  2  92  11]
 [  0  28 137]]
              precision    recall  f1-score   support

       covid      0.971     0.733     0.835        90
   pneumonia      0.697     0.876     0.776       105
     regular      0.856     0.830     0.843       165

    accuracy                          0.819       360
   macro avg      0.841     0.813     0.818       360
weighted avg      0.838     0.819     0.822       360

[[4 1 1]
 [0 6 1]
 [0 2 9]]
              precision    recall  f1-score   support

       covid      1.000     0.667     0.800         6
   pneumonia      0.667     0.857     0.750         7
     regular      0.818     0.818     0.818        11

    accuracy                          0.792        24
   macro avg      0.828     0.781     0.789        24
weighted avg      0.819     0.792     0.794        24

0.68 accuracy with a standard devi